# Reusing and bundling

To standardize our environment, we follow this road:

- define reusable and versioned components (eg on github)
- `$ref`erence reusable components
- eventualy create bundles (ewg. ith `openapi_resolver`)


### Exercise: replacing definitions with $refs

Starting from a complete file [ex-05-01-bundle.yaml](ex-05-01-bundle.yaml)
replace as many definitions as possible with references from the shared  [definitions.yaml](https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml).



In [1]:
# Exercise: creating a bundle from a $ref file
# 
# You can resolve dependencies and create a bundle file with
!pip install openapi_resolver


     |████████████████████████████████| 51kB 1.1MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/20/84/09/8338c1e5497d0ba36029ff2936afbf537b47b1bed7f8984196
  Stored in directory: /root/.cache/pip/wheels/f9/b2/4a/68efdfe5093638a9918bd1bb734af625526e849487200aa171
Successfully built openapi-resolver pathlib


In [3]:
# Exercise: create a bundle from the previous file with
!python -m openapi_resolver /code/notebooks/oas3/ex-05-01-bundle.yaml

openapi: 3.0.0
info:
  title: foo
  version: '0.1'
servers:
- description: Development server
  url: https://localhost:8443/datetime/v1
- description: Test server
  url: https://api.example.com/datetime/v1
  x-healthCheck:
    interval: 300
    timeout: 15
    url: https://api.example.com/datetime/v1/status
  x-sandbox: true
paths:
  /status:
    get:
      description: |
        Ritorna lo stato dell'applicazione. A scopo
        di test, su base randomica puo' ritornare
        un errore.
      operationId: api.get_status
      responses:
        '200':
          content:
            application/problem+json:
              example:
                detail: The app is running smoothly.
                status: 200
                title: OK
              schema:
                $ref: '#/components/schemas/Problem'
          description: |
            Il server ha ritornato lo status. In caso di problemi
            ritorna sempre un problem+json.
        '503':
          $ref: '#/compone

### YAML anchors are your friends

YAML has a very nice feature, named **anchors**. They allows to define and reference
given portions of a YAML file.

```
# the following &anchor stores the `foo` value 
a: &this_is_an_anchor foo

# *star dereferences the anchor
b: *this_is_an_anchor

```

See [anchors.yaml](/edit/notebooks/anchors.yaml)

In [12]:
#  Check yaml file content.
from pathlib import Path

content = Path('anchors.yaml').read_text()

print(content)

#
# YAML examples from https://learnxinyminutes.com/docs/yaml/
#

anchored_content: &anchor_name This string will appear as the value of two keys.
other_anchor: *anchor_name

# Anchors can be used to duplicate/inherit properties
base: &base
  name: Everyone has same name

# The regexp << is called Merge Key Language-Independent Type. It is used to
# indicate that all the keys of one or more specified maps should be inserted
# into the current map.

foo: &foo
  <<: *base
  age: 10

bar: &bar
  <<: *base
  age: 20




In [20]:
ret = yaml.safe_load(content)

assert ret['anchored_content'], ret['other_anchor']

print(ret['foo'])
print(ret['bar'])

{'name': 'Everyone has same name', 'age': 10}
{'name': 'Everyone has same name', 'age': 20}


### Using YAML anchors in OAS3

As every operation may have a set of predefined response, namely:

- 503 Service Unavailable
- 429 Too Many Requests

You can put them in an `x-` custom parameter.which will be ignored by the OAS spec parser. 

```
x-common-responses: &common-responses
  503ServiceUnavailable:
    $ref: ... 
  429TooManyRequests:
    $ref: ...

```

Then use the anchor to reference them.

```
paths:
  /status:
    get:
      ...
      responses:
        # Add the anchored responses
        <<: *common-responses
        
        # And now the other ones.
        '200':
          ...
```

## NOTE: ANCHORS ARE PROCESSED BY THE YAML PARSER, NOT BY OAS
## OAS knows nothing about ANCHORS